In [1]:
# Remove padding around screen
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [2]:
from os import walk
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy
import keras
import PIL
from PIL import Image

output_angles = []
angle = -25
for i in range(45):
    output_angles.append(round(angle,2))
    angle+=1.155
    
def parse_img_name(fstr):
    data = fstr.split("_")
    if(len(data)>2):
        return(data[3])
    else:
        return -1

def prepare_files(path):
    # Numpy array containing images
    imgs = []
    # List containing steering angles
    steerings = []
    files = get_image_paths(path)
    # Prepare training data
    count = 0
    for fname in files:
        count+=1
        #if(count > 500):    # Use this to limit loaded images for testing
        #    break
        parsed = parse_img_name(fname)
        if not(parsed == -1): # Make sure the file is valid
            # steerings.append( parse_img_name(fname) ) # Append steering
            ohe_steering = one_hot_steering(parse_img_name(fname))
            if(ohe_steering == None):
                continue # Steering is not valid, continue without bad data
            steerings.append( ohe_steering )
            img = load_image(path+fname)    # Load the image
            imgs.append(img)                  # Append image to images array
    return imgs, steerings

def get_image_paths(path):
    # Walk through all the images in the folder path
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
    return f

def load_image(infilename):
    size = 64,60
    img = Image.open(infilename)
    img.thumbnail(size, Image.ANTIALIAS)
    data = numpy.asarray(img,dtype="uint8")
    return data

def channelSplit(image):
    return numpy.dsplit(image,image.shape[-1])

def one_hot_steering(angle):
    weights=[1,1,2,3,4,5,4,3,2,1,1]  # Weights to output feedback unit
    encoding = [0] * 45
    for i in range(len(output_angles)-1):
        if (output_angles[i] <= float(angle)) and (float(angle) <= output_angles[i+1]):
            for j in range(i-5,i+5,1): # Iterate through encoding and place weights
                if(j>=0 and j<(len(encoding))):
                    encoding[j] = weights[j+5-i]
            return encoding
        if (i == len(output_angles)-1):
            print("Data incorrectly formatted")
            return encoding
            break



/home/jason/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
filepath = "../datasets/track_sess_2-23/"
imgs, steerings = prepare_files(filepath)
train_y = numpy.array(steerings)
train_x = numpy.array(imgs)
# AC-frame_011922_st_23.87382_th_0.3.jpg
# one_hot_steering("24")
train_y.shape[1:]

(45,)

In [5]:
model = keras.models.load_model('../outputs/alvinnw3.h5')

# Load callbacks
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath='../outputs/alvinnw3.h5',
    verbose=1, 
    save_best_only=True)


model.summary()
model.compile(optimizer='SGD',
              loss='logcosh',
              metrics=['accuracy'])
history = model.fit(x=train_x,y=train_y,epochs=10,validation_split=0.1,callbacks=[checkpointer])


ResourceExhaustedError: OOM when allocating tensor with shape[84480,500] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_1_1/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=9383381, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_1_1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_1_1/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jason/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jason/tensorflow/lib/python3.6/site-packages/traitlets/config/application